In [1]:
import pandas as pd
import numpy as np
import scipy
import scanpy as sc

import matplotlib.pyplot as plt
import seaborn as sns

import os, sys, shutil, importlib, glob
from tqdm.notebook import tqdm

from celloracle import motif_analysis as ma
import celloracle as co
co.__version__


'0.20.0'

In [2]:
os.chdir('/home/kl467102/thesis/')
cicero_output = "results/cicero_output/pbmc10k"
dataset_name = "pbmc10k"
celloracle_dir = f"results/celloracle_output/{dataset_name}"


## Load scATAC peak data and peak connection data made with Cicero

In [3]:
# Load scATAC-seq peak list.
peaks = pd.read_csv(cicero_output+"/all_peaks.csv", index_col=0)
peaks = peaks.x.values
peaks

array(['chr1_9790_10675', 'chr1_180599_181702', 'chr1_191168_192093', ...,
       'KI270713.1_29624_30457', 'KI270713.1_31340_32243',
       'KI270713.1_36927_37836'], dtype=object)

In [4]:
# Load Cicero coaccessibility scores.
cicero_connections = pd.read_csv(cicero_output+"/cicero_connections.csv", index_col=0)
cicero_connections.head()

,Peak1,Peak2,coaccess
1,GL000194.1_100988_101878,GL000194.1_27943_28829,0.0
2,GL000194.1_100988_101878,GL000194.1_55799_56585,0.0
4,GL000194.1_100988_101878,GL000194.1_114521_115378,0.0
5,GL000194.1_114521_115378,GL000194.1_27943_28829,0.0
6,GL000194.1_114521_115378,GL000194.1_55799_56585,0.0


## Annotate transcription start sites

In [5]:
tss_annotated = ma.get_tss_info(peak_str_list=peaks, ref_genome="hg38") 

# Check results
tss_annotated.tail()


que bed peaks: 143887
tss peaks in que: 21453


***** WARNING: File /tmp/pybedtools.qvw8292y.tmp has inconsistent naming convention for record:
KI270728.1	1791263	1791986

***** WARNING: File /tmp/pybedtools.qvw8292y.tmp has inconsistent naming convention for record:
KI270728.1	1791263	1791986



,chr,start,end,gene_short_name,strand
21448,chr1,10430108,10431038,CENPS-CORT,+
21449,chr5,149549620,149550343,CSNK1A1,-
21450,chr5,149551094,149552005,CSNK1A1,-
21451,chr20,10673806,10674617,JAG1,-
21452,chr9,122228215,122229048,LHX6,-


## Integrate TSS info and cicero connections

In [6]:
integrated = ma.integrate_tss_peak_with_cicero(tss_peak=tss_annotated, 
                                               cicero_connections=cicero_connections)
print(integrated.shape)
integrated.head()


(496245, 3)


,peak_id,gene_short_name,coaccess
0,chr10_100009458_100010353,BLOC1S2,0.002734
1,chr10_100009458_100010353,CHUK,0.046883
2,chr10_100009458_100010353,CWF19L1,0.060294
3,chr10_100009458_100010353,DNMBP,1.000000
4,chr10_100009458_100010353,ERLIN1,0.163401


### filter peaks

In [7]:
peak = integrated[integrated.coaccess >= 0.8]
peak = peak[["peak_id", "gene_short_name"]].reset_index(drop=True)
print(peak.shape)
peak.head()


(21256, 2)


,peak_id,gene_short_name
0,chr10_100009458_100010353,DNMBP
1,chr10_100185609_100186527,ERLIN1
2,chr10_100229090_100229936,CHUK
3,chr10_100267214_100268053,CWF19L1
4,chr10_100286203_100287049,BLOC1S2


In [8]:
peaks_path = cicero_output+"/peak_gene_list.csv"
peak.to_csv(peaks_path)


In [ ]:
ref_genome = 'hg19'
genome_installation = ma.is_genome_installed(ref_genome=ref_genome,
                                             genomes_dir=None)
print(ref_genome, "installation: ", genome_installation)


genome hg19 is not installed in this environment.
Please install genome using genomepy.
e.g.
    >>> import genomepy
    >>> genomepy.install_genome(name="hg19", provider="UCSC")
hg19 installation:  False


In [ ]:
if not genome_installation:
    import genomepy
    genomepy.install_genome(name=ref_genome, provider="UCSC", genomes_dir=None)
else:
    print(ref_genome, "is installed.")


17:40:12 | INFO | Downloading assembly summaries from UCSC
17:40:17 | INFO | Downloading genome from UCSC. Target URL: https://hgdownload.soe.ucsc.edu/goldenPath/hg19/bigZips/chromFa.tar.gz...


Download:   0%|          | 0.00/905M [00:00<?, ?B/s]

17:41:33 | INFO | Genome download successful, starting post processing...
17:42:28 | INFO | name: hg19
17:42:28 | INFO | local name: hg19
17:42:28 | INFO | fasta: /home/kl467102/.local/share/genomes/hg19/hg19.fa


Filtering Fasta: 0.00 lines [00:00, ? lines/s]

In [ ]:
peaks = pd.read_csv(peaks_path, index_col=0)
peaks.head()


,peak_id,gene_short_name
0,chr10_100027083_100030768,LOXL4
1,chr10_100174462_100175385,PYROXD2
2,chr10_100205554_100206987,HPS1
3,chr10_100205554_100206987,LOC101927278
4,chr10_100995978_100996580,HPSE2


In [ ]:
peaks = ma.check_peak_format(peaks, ref_genome, genomes_dir=None)


Peaks before filtering:  20396
Peaks with invalid chr_name:  0
Peaks with invalid length:  3
Peaks after filtering:  20393


## Instantiate TFinfo object and search for TF binding motifs

In [ ]:
tfi = ma.TFinfo(peak_data_frame=peaks, 
                ref_genome=ref_genome,
                genomes_dir=None) 


In [ ]:
os.makedirs(celloracle_dir, exist_ok=True)

In [ ]:
# Scan motifs. !!CAUTION!! This step may take several hours if you have many peaks!
tfi.scan(fpr=0.02, 
         motifs=None,  # If you enter None, default motifs will be loaded.
         verbose=True)

# Save tfinfo object
hdf5_path = celloracle_dir+"/test1.celloracle.tfinfo"
tfi.to_hdf5(file_path=hdf5_path)



No motif data entered. Loading default motifs for your species ...
 Default motif for vertebrate: gimme.vertebrate.v5.0. 
 For more information, please see https://gimmemotifs.readthedocs.io/en/master/overview.html 

Initiating scanner... 



2025-04-15 17:51:01,516 - DEBUG - using background: genome hg19 with size 200


Calculating FPR-based threshold. This step may take substantial time when you load a new ref-genome. It will be done quicker on the second time. 



2025-04-15 17:51:09,318 - DEBUG - determining FPR-based threshold


Motif scan started .. It may take long time.



Scanning:   0%|          | 0/17784 [00:00<?, ? sequences/s]

In [ ]:
tfi.scanned_df.head()


,seqname,motif_id,factors_direct,factors_indirect,score,pos,strand
0,chr10_100027083_100030768,GM.5.0.Mixed.0001,,"EGR1, SRF",8.491542,1078,1
1,chr10_100027083_100030768,GM.5.0.Mixed.0001,,"EGR1, SRF",8.369501,1473,-1
2,chr10_100027083_100030768,GM.5.0.Mixed.0001,,"EGR1, SRF",8.280041,1474,-1
3,chr10_100027083_100030768,GM.5.0.Mixed.0001,,"EGR1, SRF",8.280041,1475,-1
4,chr10_100027083_100030768,GM.5.0.Mixed.0001,,"EGR1, SRF",8.280041,1476,-1


## Filtering motifs


In [ ]:
# Reset filtering 
tfi.reset_filtering()

# Do filtering
tfi.filter_motifs_by_score(threshold=10)

# Format post-filtering results.
tfi.make_TFinfo_dataframe_and_dictionary(verbose=True)


Filtering finished: 19544277 -> 3733249
1. Converting scanned results into one-hot encoded dataframe.


  0%|          | 0/17752 [00:00<?, ?it/s]

2. Converting results into dictionaries.


  0%|          | 0/18531 [00:00<?, ?it/s]

  0%|          | 0/1094 [00:00<?, ?it/s]

In [ ]:
df = tfi.to_dataframe()
df.head()


,peak_id,gene_short_name,9430076C15RIK,AC002126.6,AC012531.1,AC226150.2,AFP,AHR,AHRR,AIRE,...,ZNF784,ZNF8,ZNF816,ZNF85,ZSCAN10,ZSCAN16,ZSCAN22,ZSCAN26,ZSCAN31,ZSCAN4
0,chr10_100027083_100030768,LOXL4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,chr10_100174462_100175385,PYROXD2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,chr10_100205554_100206987,HPS1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,chr10_100205554_100206987,LOC101927278,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,chr10_100995978_100996580,HPSE2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df = tfi.to_dataframe()
parquet_path = celloracle_dir+"/tfinfo.parquet"
df.to_parquet(parquet_path)


In [ ]:
from celloracle.utility import save_as_pickled_object

# If you want, you can save the result as a dictionary as follows.
td = tfi.to_dictionary(dictionary_type="TF2targetgenes")
save_as_pickled_object(td, celloracle_dir+"/TFinfo_TF2targetgenes.pickled")


In [ ]:
tf_df = df.copy()

## PROCESS

In [ ]:
dataset_name = "kim23-hm1_STRING12_3000TFs-"
celloracle_dir = f"results/celloracle_output/{dataset_name}"
os.makedirs(celloracle_dir, exist_ok=True)


In [ ]:
# Load data.
adata = sc.read_h5ad(f"/home/kl467102/Beeline-238/inputs/kim23/{dataset_name}.h5ad")
adata


AnnData object with n_obs × n_vars = 7652 × 3000
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'n_genes', 'n_counts', 'leiden'
    var: 'gene_ids', 'feature_types', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'draw_graph', 'hvg', 'leiden', 'leiden_colors', 'leiden_sizes', 'log1p', 'neighbors', 'paga', 'pca', 'umap'
    obsm: 'X_draw_graph_fr', 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [ ]:
print(f"Cell number is :{adata.shape[0]}")
print(f"Gene number is :{adata.shape[1]}")

Cell number is :7652
Gene number is :3000


In [ ]:
oracle = co.Oracle()


In [ ]:
# Check data in anndata
print("Metadata columns :", list(adata.obs.columns))
print("Dimensional reduction: ", list(adata.obsm.keys()))


Metadata columns : ['n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'n_genes', 'n_counts', 'leiden']
Dimensional reduction:  ['X_draw_graph_fr', 'X_pca', 'X_umap']


In [ ]:
# In this notebook, we use the unscaled mRNA count for the nput of Oracle object.
adata.X = adata.layers["counts"].copy()

# Instantiate Oracle object.
oracle.import_anndata_as_raw_count(adata=adata,
                                   cluster_column_name="leiden",
                                   embedding_name="X_umap")


In [ ]:
w1 = adata.X.toarray()
w1[0:5, 0:5]

array([[0., 0., 0., 0., 4.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 8.],
       [0., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
oracle.import_TF_data(TF_info_matrix=tf_df)

In [ ]:
# Perform PCA
oracle.perform_PCA()

# Select important PCs
plt.plot(np.cumsum(oracle.pca.explained_variance_ratio_)[:100])
n_comps = np.where(np.diff(np.diff(np.cumsum(oracle.pca.explained_variance_ratio_))>0.002))[0][0]
plt.axvline(n_comps, c="k")
plt.show()
print(n_comps)

28


In [ ]:
n_comps = min(n_comps, 50)


In [ ]:
n_cell = oracle.adata.shape[0]
print(f"cell number is :{n_cell}")


cell number is :7652


In [ ]:
k = int(0.025*n_cell)
print(f"Auto-selected k is :{k}")


Auto-selected k is :191


In [ ]:
oracle.knn_imputation(n_pca_dims=n_comps, k=k, balanced=True, b_sight=k*8,
                      b_maxl=k*4, n_jobs=16)


In [ ]:
oracled_path = celloracle_dir+"/withatac.celloracle.oracle"

In [ ]:
oracle.to_hdf5(oracled_path)

In [ ]:
oracle = co.load_hdf5(oracled_path)

In [ ]:
oracle.adata

AnnData object with n_obs × n_vars = 7652 × 3000
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'n_genes', 'n_counts', 'leiden'
    var: 'gene_ids', 'feature_types', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'symbol', 'isin_top1000_var_mean_genes', 'isin_TFdict_targets', 'isin_TFdict_regulators'
    uns: 'draw_graph', 'hvg', 'leiden', 'leiden_colors', 'leiden_sizes', 'log1p', 'neighbors', 'paga', 'pca', 'umap'
    obsm: 'X_draw_graph_fr', 'X_pca', 'X_umap'
    var

In [ ]:
%%time
# This step may take some time.(~30 minutes)
links = oracle.get_links(cluster_name_for_GRN_unit="leiden", alpha=10,
                         verbose_level=10)


  0%|          | 0/12 [00:00<?, ?it/s]

Inferring GRN for 0...


  0%|          | 0/2119 [00:00<?, ?it/s]

Inferring GRN for 1...


  0%|          | 0/2119 [00:00<?, ?it/s]

...

Inferring GRN for 8...


  0%|          | 0/2119 [00:00<?, ?it/s]

Inferring GRN for 9...


  0%|          | 0/2119 [00:00<?, ?it/s]

CPU times: user 49min 40s, sys: 1min 32s, total: 51min 12s
Wall time: 54min 47s


In [ ]:
links.links_dict.keys()


dict_keys(['0', '1', '10', '11', '2', '3', '4', '5', '6', '7', '8', '9'])

In [ ]:
grns_path = celloracle_dir +'/grns_ATAC/'
os.makedirs(grns_path, exist_ok=1)


In [ ]:
grns_path

'results/celloracle_output/kim23-hm1_STRING12_3000TFs-/grns_ATAC/'

In [ ]:
for key in links.links_dict.keys():
    links.links_dict[key].to_csv(f"{grns_path}/raw_GRN_for_{key}.csv")


In [ ]:
from pathlib import Path

directory = Path(grns_path)
files = [f for f in directory.iterdir() if f.is_file()]
df = pd.concat([pd.read_csv(f, index_col=0) for f in files], ignore_index=True)
df = df.loc[df.groupby(['source', 'target'])['coef_abs'].idxmax()]
df = df[['source', 'target', 'coef_abs'] ]
df.sort_values(by='coef_abs', inplace=True, ascending = False)
df.reset_index(drop=True, inplace=True)
df.columns = ['Gene1','Gene2','EdgeWeight']
df.to_csv(celloracle_dir + "/ATAC_rankedEdges.csv", index=False, sep = '\t')




In [ ]:
celloracle_dir

'results/celloracle_output/kim23-hm1_STRING12_3000TFs-'

In [ ]:
import os
import shutil

co_n = [1000,2000,3000]

for n in co_n:
    co_n_str = str(n)
    celloracle_output = 'results/celloracle_output'
    dataset_name = f"kim23-hm1_STRING12_{n}TFs-"
    out_dir = f"/home/kl467102/Beeline-238/outputs/kim23/{dataset_name[:-1]}/CELLORACLE_ATAC" #/CELLORACLE
    os.makedirs(out_dir, exist_ok=True)
    rankedEdges_file = f"{dataset_name}/ATAC_rankedEdges.csv"
    rankedEdges_path = os.path.join(celloracle_output, rankedEdges_file)
    # Path to the network file (assumed to be located somewhere else)
    if os.path.exists(rankedEdges_path):
        shutil.copy(rankedEdges_path, os.path.join(out_dir, 'rankedEdges.csv'))
        print(f"{rankedEdges_path} copied to {out_dir}")
    else:
        print(f"Warning: rankedEdges file not found for {rankedEdges_path}")





# rankedEdges_file = f"{dataset_name}_CellOracleRNA_rankedEdges.txt"
# rankedEdges_path = os.path.join(celloracle_output, rankedEdges_file)

results/celloracle_output/kim23-hm1_STRING12_1000TFs-/ATAC_rankedEdges.csv copied to /home/kl467102/Beeline-238/outputs/kim23/kim23-hm1_STRING12_1000TFs/CELLORACLE_ATAC
results/celloracle_output/kim23-hm1_STRING12_2000TFs-/ATAC_rankedEdges.csv copied to /home/kl467102/Beeline-238/outputs/kim23/kim23-hm1_STRING12_2000TFs/CELLORACLE_ATAC
results/celloracle_output/kim23-hm1_STRING12_3000TFs-/ATAC_rankedEdges.csv copied to /home/kl467102/Beeline-238/outputs/kim23/kim23-hm1_STRING12_3000TFs/CELLORACLE_ATAC


In [ ]:
auroc_csv = '/home/kl467102/Beeline-238/outputs/kim23/kim23/mine_AUROC.csv'
auprcratio_csv = '/home/kl467102/Beeline-238/outputs/kim23/kim23/mine_AUPRCRatio.csv'

In [ ]:
import pandas as pd

In [ ]:
auroc_df = pd.read_csv(auroc_csv)
auprcratio_csv = pd.read_csv(auprcratio_csv)

In [ ]:
auroc_df

,Unnamed: 0,kim23-dm1_STRING12_1000TFs,kim23-dm1_STRING12_500TFs,kim23-dm2_STRING12_1000TFs,kim23-dm2_STRING12_500TFs,kim23-hm1-extgenes1000,kim23-hm1-extgenes500,kim23-hm1-hm2_STRING12_500TFs,kim23-hm1_STRING12_1000TFs,kim23-hm1_STRING12_500TFs,kim23-hm2_STRING12_1000TFs,kim23-hm2_STRING12_500TFs,kim23-hm1-hm2_STRING12_1000TFs,kim23-hm1_STRING12_2000TFs,kim23-hm1_STRING12_3000TFs
0,CELLORACLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.508473,NaN,NaN,NaN,NaN,0.506208,0.505401
1,CELLORACLE_ATAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.512043,NaN,NaN,NaN,NaN,0.509089,0.508610
2,GENIE3,0.737196,0.742809,0.731839,0.742869,0.610406,0.620619,0.748343,0.759566,0.778210,0.751118,0.772496,0.736555,NaN,NaN
3,GRNBOOST2,0.726634,0.736746,0.722200,0.745256,0.602103,0.624820,0.755444,0.752445,0.779888,0.742429,0.773995,0.733719,NaN,NaN
4,PIDC,0.763450,0.770820,0.749111,0.758717,NaN,NaN,0.776643,0.785053,0.811085,0.769047,0.794975,0.759083,NaN,NaN
